In [1]:
import os

import random

## Get the Kaggle Data

### Run the Kaggle CLI tool and collect Cats vs Dogs

In [2]:
if 'data' != os.getcwd().split('/')[-1]:
    !mkdir data
    os.chdir('data')

if 'dogscats' != os.getcwd().split('/')[-1]:
    !mkdir dogscats
    os.chdir('dogscats')    

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘dogscats’: File exists


In [3]:
!pip install kaggle-cli

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
# run config for kaggle CLI
#!kg config -g -u username -p password -c dogs-vs-cats-redux-kernels-edition

In [5]:
!kg download

Starting new HTTPS connection (1): www.kaggle.com
downloading https://www.kaggle.com/c/imagenet-object-detection-challenge/download/imagenet_object_detection_test.tar.gz

Starting new HTTPS connection (1): storage.googleapis.com
imagenet_object_detection_test.tar.gz 100% |#########| Time: 0:00:14  30.4 MiB/s
downloading https://www.kaggle.com/c/imagenet-object-detection-challenge/download/imagenet_object_detection_train.tar.gz

imagenet_object_detection_train.tar.gz   0% |         | ETA: 0:35:51  26.9 MiB/s^CTraceback (most recent call last):
  File "/home/ubuntu/anaconda2/bin/kg", line 11, in <module>
    sys.exit(main())
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/kaggle_cli/main.py", line 21, in main
    return app.run(argv)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/cliff/app.py", line 279, in run
    result = self.run_subcommand(remainder)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/cliff/app.py", line 400, in run_subcommand
    result

In [6]:
%%capture
!unzip train.zip

In [7]:
%%capture
!unzip test.zip

## Split Train into Cats and Dogs

In [8]:
!mkdir train/cats
!mkdir train/dogs

mkdir: cannot create directory ‘train/cats’: No such file or directory
mkdir: cannot create directory ‘train/dogs’: No such file or directory


In [9]:
!mv train/cat.* train/cats

mv: cannot stat ‘train/cat.*’: No such file or directory


In [10]:
!mv train/dog.* train/dogs

mv: cannot stat ‘train/dog.*’: No such file or directory


## Validation set prep

For this, we want to remove a set of $n$ images from the kaggle provided image sets, and set them aside for model and parameter validation. We do this for both the cats and dogs set of images.

In [11]:
!mkdir valid

mkdir: cannot create directory ‘valid’: File exists


In [12]:
n = 1000

### Cats

In [13]:
!mkdir valid/cats

mkdir: cannot create directory ‘valid/cats’: File exists


List the cat files

In [14]:
cats = !ls train/cats
cats[:5]

['ls: cannot access train/cats: No such file or directory']

Sample the cat files

In [15]:
random.sample(cats, n)[:5]

ValueError: sample larger than population

Verify 1000 cat images are in valid/cats. If there aren't, move 1000 over to it.

In [ ]:
# bash command found here 
# https://stackoverflow.com/questions/11307257/is-there-a-bash-command-which-counts-files

num_cat_images, = !ls -1q valid/cats | wc -l
num_cat_images = int(num_cat_images)
num_cat_images

In [ ]:
if num_cat_images == 0:
    for file_ in random.sample(cats, n):
        !mv train/cats/$file_ valid/cats/$file_
elif num_cat_images != n:
    raise ValueError(' '.join(['Unexpected number of cats in valid/cats folder.', 
                               '\nExpected', str(n), 'Got', str(num_cat_images)])
                    )

Verify 1000 are removed from train/cats. The original set of cat images contained 12500 cat images.

In [ ]:
!ls -1q train/cats/* | wc -l

### Dogs

In [ ]:
!mkdir valid/dogs

List, move and verify dog file moves

In [ ]:
num_dog_images, = !ls -1q valid/dogs | wc -l
num_dog_images = int(num_dog_images)
num_dog_images

In [ ]:
dogs = !ls train/dogs

if num_dog_images == 0:
    for file_ in random.sample(dogs, n):
        !mv train/dogs/$file_ valid/dogs/$file_
elif num_dog_images != n:
    raise ValueError(' '.join(['Unexpected number of dogs in valid/dogs folder.', 
                               '\nExpected', str(n), 'Got', str(num_cat_images)])
                     )

In [ ]:
!ls -1q train/dogs/* | wc -l

### Sample set prep

For this, we want $m$ images from training, which we use to interatively train our models. The smaller set allows us to move much quicker. These images can just be copied from the training set. 

In [ ]:
to_make_folders = ['sample', 'sample/train', 'sample/train/cats', 'sample/train/dogs', 'sample/test']

for folder in to_make_folders:
    !mkdir $folder

In [ ]:
m = 100

In [ ]:
for animal in ['dogs', 'cats']:
    sample_count, = !ls -1q sample/train/$animal | wc -l
    train_count, = !ls -1q train/$animal | wc -l
    
    if int(sample_count) == 0:
        animals = !ls train/$animal

        for file_ in random.sample(animals, m):
            !cp train/$animal/$file_ sample/train/$animal/$file_
    elif int(sample_count) != m:
        raise ValueError(' '.join(['Unexpected number of', animal, 'in sample folder.', 
                               '\nExpected', str(m), 'Got', str(sample_count)])
                         )
    
    print(animal, sample_count, train_count)

In [ ]:
sample_test_count, = !ls -1q sample/test | wc -l

if int(sample_test_count) == 0:
    test_animals = !ls test

    for file_ in random.sample(test_animals, 2*m):
        !cp test/$file_ sample/test/$file_
        
elif int(sample_test_count) != 2*m:
    raise ValueError(' '.join(['Unexpected number of test images in sample/test folder.', 
                           '\nExpected', str(2*m), 'Got', str(sample_test_count)])
                     )
print(sample_test_count)